##### MAIN OVERVIEW:
##### Written 2023/08 by Christina Lebonville
##### Last updated: 2023/08/10
###### For batch processing each model's inference on all videos. 

###### Can run in this notebook but will not be able to see the progress bar. To be able to monitor the progress of the inference, you will need to go into Anaconda and activate the DEG environment like you were going to open the GUI, but instead of typing "deepethogram" to open the GUI, type "python" to allow the command line to accept the Python code below. 

###### REQUIRED KERNEL: "deg"

- Opens DEG Project & required modules
- Creates, checks, and ammends config file used for inference
- Runs inference on all videos
- Checks random file to make sure inference was successful
- MUST use "python -m ipykernel install --user --name=deg" in anaconda command line to make the deg environment detectable in the notebook and then select the environment in the notebook before running the below code

In [6]:
import deepethogram
import logging
import multiprocessing
import os
import random
import h5py
from deepethogram import configuration, postprocessing, projects, utils, file_io
from deepethogram.debug import print_dataset_info
from deepethogram.feature_extractor.inference import feature_extractor_inference
from deepethogram.sequence.inference import sequence_inference
from omegaconf import OmegaConf
import pandas as pd
import torch

config_file='project_config.yaml'
data_path='H://TrueBout-Gen4_deepethogram/DATA'
project_path='H://TrueBout-Gen4_deepethogram/'


files = os.listdir(project_path)
assert 'DATA' in files, 'DATA directory not found! {}'.format(files)
assert 'models' in files, 'models directory not found! {}'.format(files)
assert 'project_config.yaml' in files, 'project config not found! {}'.format(files)

print_dataset_info(os.path.join(project_path, 'DATA'))
print('Project Loaded Successfully Unless Substantial Errors Reported Above')

3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.


Project Loaded Successfully Unless Substantial Errors Reported Above


In [8]:
###### Check the feature extractor configuration to make sure everything is correct
#### This is where you can manually sets some of the configuration fields
cfg = configuration.make_feature_extractor_inference_cfg(project_path=project_path)

cfg.feature_extractor.weights = 'latest'
cfg.flow_generator.weights = 'latest'
# make sure errors are thrown
cfg.inference.ignore_error = False

print(OmegaConf.to_yaml(cfg))


split:
  reload: true
  file: null
  train_val_test:
  - 0.8
  - 0.2
  - 0.0
compute:
  fp16: false
  num_workers: 8
  batch_size: 32
  min_batch_size: 8
  max_batch_size: 512
  distributed: false
  gpu_id: 0
  dali: false
  metrics_workers: 0
reload:
  overwrite_cfg: false
  latest: false
notes: null
log:
  level: info
augs:
  brightness: 0.25
  contrast: 0.1
  hue: 0.1
  saturation: 0.1
  color_p: 0.5
  grayscale: 0.5
  crop_size: null
  resize:
  - 224
  - 224
  dali: false
  random_resize: false
  pad: null
  LR: 0.5
  UD: 0.0
  degrees: 10
  normalization:
    'N': 29626368000
    mean:
    - 0.44525026569574694
    - 0.07219969618068768
    - 0.025669157630121873
    std:
    - 0.13295815236680175
    - 0.04349840018167546
    - 0.029149261942258725
feature_extractor:
  arch: resnet50
  fusion: average
  sampler: null
  final_bn: false
  sampling_ratio: null
  final_activation: sigmoid
  dropout_p: 0.25
  n_flows: 10
  n_rgb: 1
  curriculum: false
  inputs: both
  weights: latest

In [2]:
##Run Feature Extractor model inference!
feature_extractor_inference(cfg)
print('Success! Feature extractor model used to infer predictions on all videos.')

Success! Check your data path for updated prediction files.


In [ ]:
##Double-check a random record to make sure an entry for the feature extractor model exists
##in this case either ResNet18 (deg_f), ResNet50 (deg_m), or 3D-ResNet34 (deg_s)
# this just parses our DATA directory, to get the path to each file for each video
records = projects.get_records_from_datadir(os.path.join(project_path, 'DATA'))
animal = random.choice(list(records.keys()))
record = records[animal]

# I call the file output by inference the `outputfile` in various places in the code
outputfile = record['output']

utils.print_hdf5(outputfile)

In [ ]:
###### Check the sequence extractor configuration to make sure everything is correct
#### This is where you can manually sets some of the configuration fields

cfg = configuration.make_sequence_inference_cfg(project_path)
cfg.sequence.weights = 'latest'
cfg.inference.ignore_error = False

In [ ]:
##Run Sequence model inference!
sequence_inference(cfg)
print('Success! Sequence model used to infer predictions on all videos. Ready to proceed to batch prediction export.')

In [ ]:
##Double-check a random record to make sure an entry for the model exists - in this case tgmj

# this just parses our DATA directory, to get the path to each file for each video
records = projects.get_records_from_datadir(os.path.join(project_path, 'DATA'))
animal = random.choice(list(records.keys()))
record = records[animal]

# I call the file output by inference the `outputfile` in various places in the code
outputfile = record['output']

utils.print_hdf5(outputfile)